In [1]:
import nnts
import nnts.data
import nnts.plotting
import nnts.torch.preprocessing
import nnts.torch.models
import nnts.metrics
import nnts.torch.datasets
import nnts.loggers
import nnts.datasets
from nnts import utils
import nnts.torch.utils
import nnts.torch.trainers
import nnts.metrics

import torch
torch.set_printoptions(precision=8, sci_mode=False)
%load_ext autoreload
%autoreload 2

### Load the dataset 
You can easily load any of the Monash datasets using the `load_dataset` function.

In [2]:
df, metadata = nnts.datasets.load_dataset("tourism_monthly")

### Set the Hyperparamters 
You will need to provide an optimizer and a loss function. Other hyperparameters are optional.

In [3]:
params = utils.Hyperparams(
    optimizer=torch.optim.Adam,
    loss_fn=nnts.torch.models.deepar.distr_nll,
    batch_size=32,
    batches_per_epoch=50,
    training_method=utils.TrainingMethod.TEACHER_FORCING,
    model_file_path="logs"
)

### Create covariate features
When using a DeepAR model we recommend using lag features as covariates.

In [4]:
import nnts.lags
lag_seq = nnts.lags.get_lags_for_frequency(metadata.freq)
lag_seq = [lag - 1 for lag in lag_seq if lag > 1]
lag_processor = nnts.torch.preprocessing.LagProcessor(lag_seq)

### Data Processing
We specify how we want to split the data, what transformations we want to apply and how we want to batch and sample the data during training.  

In [5]:
context_length = metadata.context_length + max(lag_seq)
dataset_options = {
    "context_length": metadata.context_length,
    "prediction_length": metadata.prediction_length,
    "conts": [],
    "lag_seq": lag_seq,
}

trn_dl, test_dl = nnts.torch.utils.create_dataloaders(
    df,
    nnts.datasets.split_test_train_last_horizon,
    context_length,
    metadata.prediction_length,
    Dataset=nnts.torch.datasets.TimeseriesDataset,
    dataset_options=dataset_options,
    Sampler=nnts.torch.datasets.TimeSeriesSampler,
)

### Create the model
Create a Pytorch model.

In [6]:

net = nnts.torch.models.DistrDeepAR(
    nnts.torch.models.deepar.StudentTHead,
    params,
    nnts.torch.preprocessing.masked_mean_abs_scaling,
    1,
    lag_processor=lag_processor,
    scaled_features=[],
    context_length=metadata.context_length,
    cat_idx=None,
    seq_cat_idx=None,
)


### Train the model

In [7]:

trner = nnts.torch.trainers.TorchEpochTrainer(
    net,
    params,
    metadata
)
evaluator = trner.train(trn_dl)

DistrDeepAR(
  (decoder): UnrolledLSTMDecoder(
    (rnn): LSTM(16, 40, num_layers=2, batch_first=True, dropout=0.1)
  )
  (distribution): StudentTHead(
    (main): ModuleList(
      (0-2): 3 x Linear(in_features=40, out_features=1, bias=True)
    )
  )
)
saving model
Epoch 1 Train Loss: 9.474244117736816
saving model
Epoch 2 Train Loss: 9.108116149902344
saving model
Epoch 3 Train Loss: 8.635644912719727
saving model
Epoch 4 Train Loss: 8.43875503540039
saving model
Epoch 5 Train Loss: 8.384746551513672
saving model
Epoch 6 Train Loss: 8.308905601501465
saving model
Epoch 7 Train Loss: 8.168272972106934
saving model
Epoch 8 Train Loss: 8.04916763305664
saving model
Epoch 9 Train Loss: 7.945784091949463
saving model
Epoch 10 Train Loss: 7.897384166717529
saving model
Epoch 11 Train Loss: 7.868238925933838
saving model
Epoch 12 Train Loss: 7.805572032928467
Epoch 13 Train Loss: 7.834261894226074
saving model
Epoch 14 Train Loss: 7.803522109985352
saving model
Epoch 15 Train Loss: 7.79649

### Evaluate

In [8]:
y_hat, y = evaluator.evaluate(
    test_dl, metadata.prediction_length, metadata.context_length
)

test_metrics = nnts.metrics.calc_metrics(
    y_hat, y, nnts.metrics.calculate_seasonal_error(trn_dl, metadata.seasonality)
)
test_metrics

{'mse': 108923408.0,
 'abs_error': 22129120.0,
 'abs_target_sum': 166958480.0,
 'abs_target_mean': 19007.11328125,
 'mase': 1.6643701791763306,
 'mape': 0.2491537630558014,
 'smape': 0.20001694560050964,
 'msmape': 0.19999928772449493,
 'nd': 0.1964215189218521,
 'mae': 2519.253173828125,
 'rmse': 3050.3251953125,
 'seasonal_error': 1543.2191162109375}

In [9]:
nnts.plotting.plotly_forecasts_vs_actuals(y, y_hat)

### Forecast

In [10]:
net = evaluator.net

forecaster = nnts.torch.trainers.TorchForecaster(net)

In [11]:
forecast = forecaster.forecast(test_dl, metadata.prediction_length, metadata.context_length)

In [12]:
forecast

tensor([[[  6614.57519531],
         [  4157.01171875],
         [  3148.29980469],
         ...,
         [  4086.52294922],
         [  4690.41210938],
         [  7851.56738281]],

        [[193461.17187500],
         [159843.23437500],
         [154183.06250000],
         ...,
         [158364.46875000],
         [167417.09375000],
         [239604.64062500]],

        [[ 92988.10156250],
         [ 86258.26562500],
         [ 99657.33593750],
         ...,
         [128380.63281250],
         [134643.65625000],
         [114130.62500000]],

        ...,

        [[  7770.13916016],
         [ 14293.85742188],
         [ 10809.56445312],
         ...,
         [  7772.87597656],
         [  7369.18505859],
         [  9430.20019531]],

        [[  8465.18261719],
         [  8142.86865234],
         [ 12949.27929688],
         ...,
         [ 16933.46289062],
         [ 12407.38183594],
         [ 11923.59863281]],

        [[  2143.30200195],
         [  1794.69482422],
         [